In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!ls -lha kaggle.json

!pip install -q Kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

ls: cannot access 'kaggle.json': No such file or directory
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kausr25/chatterbotenglish

!unzip chatterbotenglish.zip

Dataset URL: https://www.kaggle.com/datasets/kausr25/chatterbotenglish
License(s): unknown
  0% 0.00/23.2k [00:00<?, ?B/s]
100% 23.2k/23.2k [00:00<00:00, 42.9MB/s]
Archive:  chatterbotenglish.zip
  inflating: ai.yml                  
  inflating: botprofile.yml          
  inflating: computers.yml           
  inflating: emotion.yml             
  inflating: food.yml                
  inflating: gossip.yml              
  inflating: greetings.yml           
  inflating: health.yml              
  inflating: history.yml             
  inflating: humor.yml               
  inflating: literature.yml          
  inflating: money.yml               
  inflating: movies.yml              
  inflating: politics.yml            
  inflating: psychology.yml          
  inflating: science.yml             
  inflating: sports.yml              
  inflating: trivia.yml              


In [ ]:
import yaml
import os

dir_path = '/content'
data = os.listdir(dir_path + os.sep)

In [ ]:
data

['.config',
 'food.yml',
 'politics.yml',
 'emotion.yml',
 'sports.yml',
 'gossip.yml',
 'computers.yml',
 'ai.yml',
 'literature.yml',
 'trivia.yml',
 'science.yml',
 'botprofile.yml',
 'psychology.yml',
 'humor.yml',
 'money.yml',
 'history.yml',
 'greetings.yml',
 'health.yml',
 'chatterbotenglish.zip',
 'movies.yml',
 'sample_data']

In [ ]:
files_list = []
for i in data:
  if i.endswith(('.yml', '.yaml')):
    files_list.append(i)

In [ ]:
files_list

['food.yml',
 'politics.yml',
 'emotion.yml',
 'sports.yml',
 'gossip.yml',
 'computers.yml',
 'ai.yml',
 'literature.yml',
 'trivia.yml',
 'science.yml',
 'botprofile.yml',
 'psychology.yml',
 'humor.yml',
 'money.yml',
 'history.yml',
 'greetings.yml',
 'health.yml',
 'movies.yml']

In [ ]:
questions, answers = [], []

for filepath in files_list:
    file_ = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file_)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 2 :
            questions.append(con[0])
            replies = con[1 :]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con)> 1:
            questions.append(con[0])
            answers.append(con[1])

In [ ]:
questions

['do you drink',
 'do you drink',
 'electricity',
 'Are you experiencing an energy shortage?',
 'Are you experiencing an energy shortage?',
 'Why can you not eat?',
 'If you could eat food, what would you eat?',
 'Do you wish you could eat food?',
 'can a robot get drunk?',
 'i like wine, do you?',
 'what do robots need to survive?',
 'will robots ever be able to eat?',
 'what is good to eat?',
 "why don't you eat",
 'do you eat',
 'do you eat',
 'do you eat',
 'have you read the communist',
 'what is a government',
 'what is greenpeace',
 'what is capitalism',
 'what is socialism',
 'what is government',
 'what is communism',
 'what is impeached',
 'i do not like guns',
 'i do not like guns',
 'do you like guns',
 'why guns',
 'who was the first impeached president',
 'who is the governor',
 'who is the governor',
 'guns',
 'You are arrogant',
 'You are bragging',
 'You are never sad',
 'You are jealous',
 'You are never nice',
 'You will be happy',
 'You should be ashamed',
 'You can

In [ ]:
import numpy as np

In [ ]:
from tensorflow.keras import layers , activations , models , preprocessing, utils

In [ ]:
answers_with_tags = []
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = []
for i in range(len(answers_with_tags)) :
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index)+1

In [ ]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

In [ ]:
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')
encoder_input_data = np.array(padded_questions)

In [ ]:
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_input_data = np.array(padded_answers)

In [ ]:
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)

In [ ]:
import tensorflow as tf

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax)
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 22)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 74)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 22, 200)              378800    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 74, 200)              378800    ['input_2[0][0]']             
                                                                                              

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=32, epochs=100)

Epoch 1/100
18/18 [==============================] - 1s 28ms/step - loss: 0.1858
Epoch 2/100
18/18 [==============================] - 1s 28ms/step - loss: 0.1866
Epoch 3/100
18/18 [==============================] - 0s 27ms/step - loss: 0.1851
Epoch 4/100
18/18 [==============================] - 1s 30ms/step - loss: 0.1852
Epoch 5/100
18/18 [==============================] - 1s 39ms/step - loss: 0.1807
Epoch 6/100
18/18 [==============================] - 1s 37ms/step - loss: 0.1823
Epoch 7/100
18/18 [==============================] - 1s 39ms/step - loss: 0.1793
Epoch 8/100
18/18 [==============================] - 1s 43ms/step - loss: 0.1793
Epoch 9/100
18/18 [==============================] - 1s 30ms/step - loss: 0.1760
Epoch 10/100
18/18 [==============================] - 1s 28ms/step - loss: 0.1779
Epoch 11/100
18/18 [==============================] - 0s 28ms/step - loss: 0.1753
Epoch 12/100
18/18 [==============================] - 1s 29ms/step - loss: 0.1766
Epoch 13/100
18/18 [=====

In [ ]:
encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

In [ ]:
decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding , initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.models.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
def preprocess_input(input_sentence):
    tokens = input_sentence.lower().split()
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word])
    return preprocessing.sequence.pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
tests = ['Hello', 'Are you a bot', 'What is your name', 'That is a very long name', 'see you later']

for i in range(5):
    states_values = encoder_model.predict(preprocess_input(tests[i]))
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''

    while not stop_condition :
        dec_outputs , h , c = decoder_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None

        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += f' {word}'
                sampled_word = word

        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True

        empty_target_seq = np.zeros((1 , 1))
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h , c]
    print(f'Human: {tests[i]}')
    print()
    decoded_translation = decoded_translation.split(' end')[0]
    print(f'Bot: {decoded_translation}')
    print('-'*25)

1/1 [==============================] - 0s 35ms/step
Human: Hello

Bot:  greetings
-------------------------
1/1 [==============================] - 0s 21ms/step
Human: Are you a bot

Bot:  i certainly am i shouldn't try so hard
-------------------------
1/1 [==============================] - 0s 21ms/step
Human: What is your name

Bot:  i am everywhere
-------------------------
1/1 [==============================] - 0s 20ms/step
Human: That is a very long name

Bot:  no
-------------------------
1/1 [==============================] - 0s 31ms/step
Human: see you later

Bot:  no i am not capable of feeling people are talking to me
-------------------------
